In [2]:
import polars as pl
from tqdm import tqdm
from rdkit import Chem

In [3]:
data = pl.read_csv('data/zinc/zinc15_250k.csv')
data

smiles
str
"""CCN(CCSC)C(=O)N[C@@](C)(CC)C(F…"
"""CC1(C)CN(C(=O)Nc2cc3ccccc3nn2)…"
"""CC[C@H](NC(C)=O)C(=O)NCC1(NC(=…"
"""O=C(N[C@@H]1CC[C@H](F)C1)[C@H]…"
"""COCC(=O)N(C)CC(=O)NCC1(Nc2nccn…"
…
"""Cc1ccn(C)c1C(=O)OCCc1cn(Cc2ccc…"
"""Nc1nc(Cl)cc2ncn(-c3cccc(C(=O)N…"
"""Cc1c(C(=O)N[C@H](C)Cn2cccn2)cn…"


In [4]:
from rdkit.Chem.Descriptors import CalcMolDescriptors

In [5]:
row = []
for i in tqdm(data['smiles']):
    desc = CalcMolDescriptors(Chem.MolFromSmiles(i))
    sub = {'smiles': i}
    for k, v in desc.items():
        sub[k] = v
    row.append(sub)
zinc = pl.DataFrame(row)
# zinc.to_csv('data/zinc/descriptors.csv', index = False)

100%|██████████| 250000/250000 [24:09<00:00, 172.51it/s]


AttributeError: 'DataFrame' object has no attribute 'to_csv'

In [6]:
zinc.write_csv('data/zinc/descriptors.csv')

In [7]:
zinc.head()

smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,AvgIpc,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,…,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""CCN(CCSC)C(=O)N[C@@](C)(CC)C(F…",12.847028,12.847028,0.192141,-4.448088,0.813426,15.055556,286.363,265.195,286.132669,108,0,0.410764,-0.324266,0.410764,0.324266,1.333333,1.888889,2.222222,32.166364,9.983547,2.506565,-2.33959,2.322233,-2.589096,7.981279,-0.188741,2.278945,4.689303,243.25415,14.397341,11.226031,12.042527,8.213856,5.841824,7.23567,4.18604,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
"""CC1(C)CN(C(=O)Nc2cc3ccccc3nn2)…",12.782073,12.782073,0.187749,-0.4183,0.861236,25.44,342.399,320.223,342.169191,132,0,0.32276,-0.378263,0.378263,0.32276,1.24,2.08,2.88,16.57234,9.949965,2.475944,-2.472764,2.343682,-2.6156,5.904205,-0.184603,2.998999,1.5245,773.479822,17.493353,14.488671,14.488671,11.94782,8.511913,8.511913,7.421962,…,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
"""CC[C@H](NC(C)=O)C(=O)NCC1(NC(=…",12.108206,12.108206,0.091773,-0.557541,0.592397,16.125,337.38,314.196,337.175004,132,0,0.242149,-0.351964,0.351964,0.242149,1.291667,2.0,2.5,16.563397,10.150639,2.440597,-2.206016,2.342322,-2.510526,5.865991,-0.128019,3.169638,1.812369,591.309427,17.742276,14.140305,14.140305,11.348973,7.98876,7.98876,6.180773,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""O=C(N[C@@H]1CC[C@H](F)C1)[C@H]…",13.165747,13.165747,0.015019,-0.751764,0.933437,32.590909,305.349,285.189,305.142722,118,0,0.223467,-0.486147,0.486147,0.223467,1.136364,1.954545,2.727273,19.144178,10.076906,2.447214,-2.185646,2.491714,-2.340244,5.828642,-0.122625,3.238834,1.380225,567.401084,15.104084,12.386802,12.386802,10.70351,8.044823,8.044823,6.46763,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""COCC(=O)N(C)CC(=O)NCC1(Nc2nccn…",12.034431,12.034431,0.013687,-0.25837,0.601673,14.96,348.367,328.207,348.165837,134,0,0.248258,-0.374871,0.374871,0.248258,1.44,2.24,2.96,16.467672,10.232767,2.433469,-2.203379,2.379021,-2.438934,5.847991,-0.137218,3.169754,1.534145,738.239013,17.863597,14.098261,14.098261,12.027606,7.834092,7.834092,6.087784,…,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [17]:
zinc.describe()

statistic,smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,AvgIpc,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,…,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""250000""",250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,…,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,12.019824,12.019824,0.105626,-0.733466,0.744795,20.425499,311.228764,288.792793,310.976896,120.164108,0.000036,0.269207,-0.395578,0.396862,0.267923,1.34082,2.084712,2.716911,20.948906,10.024296,2.31237,-2.270544,2.254416,-2.428359,6.229059,-0.059879,2.881516,1.852333,536.657421,15.917474,12.974477,13.190514,10.498499,7.591624,7.859077,…,0.025708,0.0,0.0,0.014752,0.01404,0.000844,0.007432,0.17776,0.054396,0.045616,0.01266,0.008476,0.003988,0.000084,0.022092,0.002036,0.055172,0.012284,0.01216,0.00008,0.00008,0.195792,0.046076,0.079752,0.0,0.130492,0.000044,0.03186,0.042112,0.02614,0.015872,0.036476,0.030632,0.0,0.027276,0.038064,0.103104
"""std""",null,1.485571,1.485571,0.115833,1.117848,0.127258,6.485662,26.156878,25.073642,26.137762,11.102986,0.006,0.05874,0.056127,0.055833,0.056174,0.141381,0.183234,0.232884,8.90973,0.15646,0.125306,0.114741,0.128309,0.124958,0.764557,0.149732,0.293337,0.434126,133.223279,1.44793,1.42112,1.310691,1.04983,1.013426,0.963176,…,0.162921,0.0,0.0,0.122533,0.11931,0.029039,0.08663,0.42672,0.230264,0.212714,0.11333,0.092759,0.063404,0.009165,0.148068,0.045342,0.256921,0.117018,0.116465,0.009381,0.009381,0.425791,0.218122,0.279385,0.0,0.353531,0.006633,0.180945,0.203869,0.160626,0.125905,0.189192,0.173729,0.0,0.164134,0.268714,0.307756
"""min""","""Bc1cnc(NCC2(O)CCOCC2)c(C(=O)OC…",2.357438,2.357438,6.6613e-16,-6.071032,0.114868,8.833333,250.125,220.19,249.032334,54.0,0.0,0.001914,-0.710601,0.204161,0.001914,0.5,0.727273,0.95,14.695003,9.287755,1.865692,-2.753016,1.66431,-2.839089,4.595897,-0.946705,1.617252,0.987,132.1173,7.723615,5.205204,7.362702,4.193376,2.447895,3.61724,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [39]:
import os
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

drop_cols = []
continuous_cols = []
categorical_cols = []
binary_cols = []

def is_regression_column(series: pl.Series) -> bool:
    # 열에 소수점 이하가 있는지 판단
    if series.dtype not in [pl.Float32, pl.Float64]:
        return False
    return (series - series.cast(pl.Int64)).abs().sum() > 0

def plot_column_distribution(df: pl.DataFrame, save_dir: str = 'data/zinc/plots'):
    os.makedirs(save_dir, exist_ok=True)
    for col in tqdm(df.columns[1:]):
        series = df[col]
        values = series.to_numpy()
        if df[col].n_unique() == 1:
            drop_cols.append(col)
            continue
        plt.figure(figsize=(6, 4))
        
        tp = ''
        
        if is_regression_column(series):
            sns.histplot(values, bins=30, kde=True, color="steelblue")
            plt.title(f"[Continuous] Histogram for '{col}'")
            continuous_cols.append(col)
            tp = 'Continuous'
        else:
            unique_values = np.unique(values)
            if len(unique_values) == 2:
                sns.histplot(values, bins=30, kde=False, color="lightcoral")
                plt.title(f"[Binary/Other] Histogram for '{col}'")
                binary_cols.append(col)
                tp = 'Binary'
            else:
                sns.countplot(x=values, palette="Set2")
                plt.title(f"[Categorical] Countplot for '{col}'")
                categorical_cols.append(col)
                tp = 'Categorical'

        plt.xlabel(col)
        plt.ylabel("Count")
        plt.tight_layout()

        # 저장 경로 구성
        filename = f"{col.replace(' ', '_')}.png"
        os.makedirs(save_dir + '/' + tp + '/', exist_ok=True)
        save_path = os.path.join(save_dir + '/' + tp + '/', filename)
        plt.savefig(save_path)
        plt.close()
        
    # columns
    print(f'Continuous ({len(continuous_cols)}):', continuous_cols)
    print(f'Categorical ({len(categorical_cols)}):', categorical_cols)
    print(f'Binary ({len(binary_cols)}):', binary_cols)
    print(f'Drop ({len(drop_cols)}):', drop_cols)
    return continuous_cols, categorical_cols, binary_cols, drop_cols

In [40]:
continuous_cols, categorical_cols, binary_cols, drop_cols = plot_column_distribution(zinc)

100%|██████████| 217/217 [02:31<00:00,  1.43it/s]

Continuous ({len(continuous_cols)}): ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_V

In [41]:
len(continuous_cols), len(categorical_cols), len(binary_cols), len(drop_cols)

(105, 94, 10, 8)

In [44]:
regression_float = continuous_cols
regression_int = [c for c in categorical_cols if 'Num' in c or 'Count' in c]
classification_col = [c for c in categorical_cols if c not in regression_int]
len(regression_int)

24

In [66]:
classification_col_filtered = []
for c in classification_col:
    cz = zinc[c]
    zeros =  cz.to_list().count(0)
    if zeros / len(cz) > 0.7:
        continue
    else:
        classification_col_filtered.append(c)
len(classification_col_filtered), classification_col_filtered

(10,
 ['fr_Ar_N',
  'fr_C_O',
  'fr_C_O_noCOO',
  'fr_NH0',
  'fr_NH1',
  'fr_Ndealkylation2',
  'fr_amide',
  'fr_aryl_methyl',
  'fr_benzene',
  'fr_ether'])

In [67]:
len(regression_float), len(regression_int), len(classification_col_filtered)

(105, 24, 10)

In [69]:
targets = {'Regression_float': regression_float, 'Regression_int': regression_int, 'Classification': classification_col_filtered}

In [70]:
import pickle
with open('data/zinc/target_cols.pkl', 'wb') as f:
    pickle.dump(targets, f)

In [71]:
import pickle
with open('data/zinc/target_cols.pkl', 'rb') as f:
    target_cols = pickle.load(f)

target_cols

{'Regression_float': ['MaxAbsEStateIndex',
  'MaxEStateIndex',
  'MinAbsEStateIndex',
  'MinEStateIndex',
  'qed',
  'SPS',
  'MolWt',
  'HeavyAtomMolWt',
  'ExactMolWt',
  'MaxPartialCharge',
  'MinPartialCharge',
  'MaxAbsPartialCharge',
  'MinAbsPartialCharge',
  'FpDensityMorgan1',
  'FpDensityMorgan2',
  'FpDensityMorgan3',
  'BCUT2D_MWHI',
  'BCUT2D_MWLOW',
  'BCUT2D_CHGHI',
  'BCUT2D_CHGLO',
  'BCUT2D_LOGPHI',
  'BCUT2D_LOGPLOW',
  'BCUT2D_MRHI',
  'BCUT2D_MRLOW',
  'AvgIpc',
  'BalabanJ',
  'BertzCT',
  'Chi0',
  'Chi0n',
  'Chi0v',
  'Chi1',
  'Chi1n',
  'Chi1v',
  'Chi2n',
  'Chi2v',
  'Chi3n',
  'Chi3v',
  'Chi4n',
  'Chi4v',
  'HallKierAlpha',
  'Ipc',
  'Kappa1',
  'Kappa2',
  'Kappa3',
  'LabuteASA',
  'PEOE_VSA1',
  'PEOE_VSA10',
  'PEOE_VSA11',
  'PEOE_VSA12',
  'PEOE_VSA13',
  'PEOE_VSA14',
  'PEOE_VSA2',
  'PEOE_VSA3',
  'PEOE_VSA4',
  'PEOE_VSA5',
  'PEOE_VSA6',
  'PEOE_VSA7',
  'PEOE_VSA8',
  'PEOE_VSA9',
  'SMR_VSA1',
  'SMR_VSA10',
  'SMR_VSA2',
  'SMR_VSA3',
  'S